In [4]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.7.1
tiktoken version: 0.9.0


In [5]:
import os 
import urllib.request 


if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

len(raw_text)

20479

In [7]:
import re 

text = "Hello, world. This, is a test."

result=re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [8]:
result=re.split(r'([,.]|\s)', text)

result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [9]:
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

len(raw_text)

20479

In [11]:
text="Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result=[item.strip() for item in result if item.strip()]
len(result)

4690

In [12]:
all_words=sorted(set(result))
vocab_size=len(all_words)
print(vocab_size)

1130


In [13]:
vocab={token:integer for integer, token in enumerate(all_words)}

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        preprocessed=[
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [15]:
tokenizer = SimpleTokenizerV1(vocab)

In [16]:
text="It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [17]:
ids=tokenizer.encode(text)

In [18]:
tokenizer.decode(ids)

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [19]:
text="Hello, do you like tea. is this-- a test?"

all_tokens=sorted(list(set(result)))

all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer, token in enumerate(all_tokens)}

len(vocab.items())

1132

In [20]:
tokenizer2=SimpleTokenizerV1(vocab)

tokenizer2.encode(text)
tokenizer2.decode(tokenizer2.encode(text))

'<|unk|>, do you like tea. is this -- a <|unk|>?'

In [21]:
import tiktoken

tiktoken.__version__


'0.9.0'

In [22]:
tokenizer=tiktoken.get_encoding("gpt2")

tokenizer.encode("Hello world")

[15496, 995]

In [23]:
tokenizer.decode(tokenizer.encode("Hello world"))

'Hello world'

In [24]:
text = ( 
    "Hello, do you like tea? <|endoftext|> fsdf sdf sdfIn the sunlit terraces"
    "of someunknownPlace."
)

tokenizer.encode(text, allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 43458,
 7568,
 264,
 7568,
 264,
 7568,
 818,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

In [25]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [26]:
enc_sample=enc_text[50:]

In [27]:
enc_sample=enc_text[50:]

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")



x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [28]:
import torch

In [29]:
torch.__version__

'2.7.1+cu126'

In [30]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



In [31]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader



In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

In [36]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

first_batch = next(data_iter)
print(first_batch)



[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]
